In [1]:
import sys
import numpy as np
import pandas as pd
import functools
print('system: ', sys.version)
print('numpy: ', np.__version__)
print('panda: ', pd.__version__)

system:  3.5.1 |Anaconda 2.4.1 (64-bit)| (default, Dec  7 2015, 15:00:12) [MSC v.1900 64 bit (AMD64)]
numpy:  1.10.1
panda:  0.17.1


In [2]:
print('Reading Price data')
price_data = pd.read_pickle('data/history_M_Indicator.pickle')
price_data['DOCS25_sft1'] = price_data['DOCS(25)'].shift(1)
price_data.columns

Reading Price data


Index(['DateTime', 'Seconds', 'Date', 'Time', 'Open', 'High', 'Low', 'Close',
       'Ticks', 'Close-Roll_Mean',
       ...
       'RSI(7)-Roll_var', 'RSI(14)-Roll_Mean', 'RSI(14)-Roll_std',
       'RSI(14)-Roll_var', 'RSI(30)-Roll_Mean', 'RSI(30)-Roll_std',
       'RSI(30)-Roll_var', 'Break_Support', 'Break_Resistance', 'DOCS25_sft1'],
      dtype='object', length=107)

In [3]:
BR = price_data[(price_data['Break_Resistance'] == True)][['DateTime',
                                                   'Open', 'High', 'Low', 'Close',
                                                   'DOCS(25)', 'DOCR(25)']]

In [14]:
br_test = price_data.head(10000).copy()

In [5]:
def check_profit_loss_SELL(df, BR):
    sl = 50
    tp = 50
    point = 0.0001
    xecn = 10
    order_price = br_test.High
    c = 0
    #print('br:', BR)
    if not BR:
        SELL = 'NO ORDER'
    else:
        SELL = ''
    while (SELL == ''):
        ind = np.where(df.Break_Resistance == True)
        
        if SELL == '':
            #print('s1--------------------------------------------')
            for i in np.nditer(ind):
                subdf = df.loc[int(i):]
                #print(subdf[['DateTime', 'High', 'Low', 'DOCS(25)', 'DOCR(25)', 'DOCS25_sft1', 
                             #'Break_Support', 'Break_Resistance']])                
                sl_sell = subdf.High.iloc[0] + (sl * point * xecn)
                #print('ORDER PRICE', subdf.High.iloc[0], 'sl_sell', sl_sell)
                # Break resistance, SELL check for profit
                p = np.where(subdf.Low <= subdf.DOCS25_sft1)[0]
                # Break resistance, SELL check for loss
                l = np.where(subdf.High > sl_sell)[0]
                #print('p', p, 'l', l)
                if (p.size > 0) & (l.size > 0):
                    if p.min() < l.min():
                        SELL = 'PROFIT'
                    if l.min() < p.min():
                        SELL = 'LOSS'
                elif (p.size > 0) & (l.size == 0):
                    SELL = 'PROFIT'
                elif (p.size == 0) & (l.size > 0):
                    SELL = 'LOSS'
                elif (p.size ==0 ) & (l.size == 0):
                    SELL = 'UNK'
                #input('Enter to continue SELL')
    c += 1
    #print('c=', c)
    return SELL

def check_profit_loss_BUY(df, BR):
    sl = 50
    tp = 50
    point = 0.0001
    xecn = 10
    order_price = br_test.High
    c = 0
    #print('br:', BR)
    if not BR:
        BUY = 'NO ORDER'
    else:
        BUY = ''
    while (BUY == ''):
        ind = np.where(df.Break_Resistance == True)        
        if BUY == '':
            #print('b1--------------------------------------------')
            for i in np.nditer(ind):
                subdf = df.loc[int(i):]
                #print(subdf[['DateTime', 'High', 'Low', 'DOCS(25)', 'DOCR(25)', 'DOCS25_sft1', 
                             #'Break_Support', 'Break_Resistance']])
                tp_buy = subdf.High.iloc[0] + (tp * point * xecn)
                sl_buy = subdf.High.iloc[0] - (sl * point * xecn)
                #print('ORDER PRICE', subdf.High.iloc[0], 'tp_buy', tp_buy, 'sl_buy', sl_buy)
                # Break resistance, BUY check for profit
                p = np.where(subdf.High >= tp_buy)[0]
                # Break resistance, BUY check for loss
                l = np.where(subdf.Low <= sl_buy)[0]
                #print('p', p, 'l', l)
                if (p.size > 0) & (l.size > 0):
                    if p.min() < l.min():
                        BUY = 'PROFIT'
                    if l.min() < p.min():
                        BUY = 'LOSS'
                elif (p.size > 0) & (l.size == 0):
                    BUY = 'PROFIT'
                elif (p.size == 0) & (l.size > 0):
                    BUY = 'LOSS'
                elif (p.size ==0 ) & (l.size == 0):
                    BUY = 'UNK'
                
    c += 1
    #print('c=', c)
    return BUY

In [13]:
len(price_data)

366111

In [15]:
%timeit br_test['BR_BUY'] = list(map(functools.partial(check_profit_loss_BUY, br_test), br_test.Break_Resistance))

1 loops, best of 3: 23min 53s per loop


In [9]:
br_test['BR_SELL'] = list(map(functools.partial(check_profit_loss_SELL, br_test), br_test.Break_Resistance))

In [14]:
br_test.head(3)

,DateTime,Seconds,Date,Time,Open,High,Low,Close,Ticks,Close-Roll_Mean,...,RSI(14)-Roll_std,RSI(14)-Roll_var,RSI(30)-Roll_Mean,RSI(30)-Roll_std,RSI(30)-Roll_var,Break_Support,Break_Resistance,DOCS25_sft1,BR_BUY,BR_SELL
0,2015-01-02 08:00:00,1420185600,2015.01.02,08:00,1.20545,1.20545,1.20514,1.20516,31,NaN,...,NaN,NaN,NaN,NaN,NaN,False,False,NaN,NO ORDER,NO ORDER
1,2015-01-02 08:01:00,1420185660,2015.01.02,08:01,1.20516,1.20521,1.20503,1.20503,31,NaN,...,NaN,NaN,NaN,NaN,NaN,True,False,1.20545,NO ORDER,NO ORDER
2,2015-01-02 08:02:00,1420185720,2015.01.02,08:02,1.20503,1.20503,1.20468,1.20473,51,NaN,...,NaN,NaN,NaN,NaN,NaN,True,False,1.20514,NO ORDER,NO ORDER


In [13]:
br_test.groupby('BR_SELL').count()

,DateTime,Seconds,Date,Time,Open,High,Low,Close,Ticks,Close-Roll_Mean,...,RSI(14)-Roll_Mean,RSI(14)-Roll_std,RSI(14)-Roll_var,RSI(30)-Roll_Mean,RSI(30)-Roll_std,RSI(30)-Roll_var,Break_Support,Break_Resistance,DOCS25_sft1,BR_BUY
BR_SELL,,,,,,,,,,,,,,,,,,,,,
NO ORDER,82,82,82,82,82,82,82,82,82,68,...,68,68,68,68,68,68,82,82,81,82
PROFIT,18,18,18,18,18,18,18,18,18,18,...,18,18,18,18,18,18,18,18,18,18


In [15]:
br_test.groupby('BR_BUY').count()

,DateTime,Seconds,Date,Time,Open,High,Low,Close,Ticks,Close-Roll_Mean,...,RSI(14)-Roll_Mean,RSI(14)-Roll_std,RSI(14)-Roll_var,RSI(30)-Roll_Mean,RSI(30)-Roll_std,RSI(30)-Roll_var,Break_Support,Break_Resistance,DOCS25_sft1,BR_SELL
BR_BUY,,,,,,,,,,,,,,,,,,,,,
NO ORDER,82,82,82,82,82,82,82,82,82,68,...,68,68,68,68,68,68,82,82,81,82
UNK,18,18,18,18,18,18,18,18,18,18,...,18,18,18,18,18,18,18,18,18,18


In [6]:
price_data['BR_BUY'] = list(map(functools.partial(check_profit_loss_BUY, price_data), price_data.Break_Resistance))
price_data['BR_SELL'] = list(map(functools.partial(check_profit_loss_SELL, price_data), price_data.Break_Resistance))

KeyboardInterrupt: 

In [11]:
?%timeit